# How to use `locals`
`locals` is a pure Python package that ingests JWST Wide-Field Slitless Spectroscopy (WFSS) data and returns a source catalog of all the low-mass stars in the field along with their calculated fundamental and secondary parameters.

In [1]:
# Imports
from locals import SourceCatalog
from sedkit import SED
import astropy.units as q
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

ImportError: No module named 'healpy'

## Making an SED for a single source (no WFSS spectra)

For each spectrum in our WFSS data products, an `SED` object is created. `locals` then searches Vizier to find ancillary photometry, parallaxes, and spectral types. Then it assembles an SED from the available data and calculates fundamental and atmospheric parameters.

Let's pretend one of the brown dwarfs in our field is [LHS2924](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=LHS2924&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id). Here is what `locals` is doing for each source under the hood.

In [2]:
# Coordinates of LHS 2924 (Usually taken from the WFSS source_list)
ra = 217.180137*q.deg
dec = 33.177539*q.deg

# Make the SED object
src = SED(ra=ra, dec=dec, name='LHS 2924', spectral_type='M9', radius=(1.06*q.R_jup,0.05*q.R_jup))

# Find some photometry
src.find_SDSS()
src.find_2MASS()
src.find_WISE()
src.find_PanSTARRS()

# Find a distance from Gaia DR2
src.find_Gaia()

# Take a look
print('\n',src.photometry,'\n')

Setting radius to (<Quantity 1.06 jupiterRad>, <Quantity 0.05 jupiterRad>)
Setting parallax to (<Quantity 90.9962 mas>, <Quantity 0.1271 mas>) and distance to (<Quantity 10.99 pc>, <Quantity 0.02 pc>, <Quantity 0. pc>).
1 record for <SkyCoord (ICRS): (ra, dec) in deg
    (217.180137, 33.177539)> found in Simbad.
Setting name to LHS 2924 and sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (217.180137, 33.177539)>
Setting age to (<Quantity 6. Gyr>, <Quantity 4. Gyr>)
169 records found within 20.0 arcsec of NLTT 37480 in SDSS.
4 records found within 20.0 arcsec of NLTT 37480 in 2MASS.
55 records found within 20.0 arcsec of NLTT 37480 in WISE.
70 records found within 20.0 arcsec of NLTT 37480 in PanSTARRS.
13 records found in Gaia DR2.
Setting parallax to (<Quantity 90.9962 mas>, <Quantity 0.1271 mas>) and distance to (<Quantity 10.99 pc>, <Quantity 0.02 pc>, <Quantity 0. pc>).

   band     eff   ...      ref      
            um   ...               
-------- ------- ... -------------

Now we can calculate the fundamental parameters and plot the results.

In [3]:
src.results
# 2275K from Fili15

param,value,unc,units
object,object,object,object
Lbol,1.03e+30,1.02e+29,erg / s
Lbol_sun,-3.57,0.043,--
Mbol,13.679,0.108,--
SpT,M9V,--,--
SpT_fit,--,--,--
Teff,2240,76,K
Teff_bb,--,--,--
Teff_evo,2334.995246583482,88.54356084564915,K
abs_spec_SED,--,--,--


Let's fit a blackbody for fun.

In [48]:
src.fit_blackbody()


Blackbody fit: 1831 K


In [4]:
fig = src.plot(integral=True)

## Make SEDs for all sources in the WFSS field
Now that we see the output of a single source, we can run this on the entire `SourceCatalog` generated from the WFSS output. The difference here is that a WFSS spectrum will be included in each `Source`.

Right now, we are only interested in brown dwarfs so we will include a color cut to only process sources within brown dwarf color space.

In [5]:
# Path to the JWST pipeline output directory
cat_path = resource_filename('locals', 'data/fake/')

# Calculate all sources in the pipeline output directory
cat = SourceCatalog(cat_path, color_cut='brown_dwarfs')

UnitConversionError: 'deg' (angle) and 'h' (time) are not convertible

All the sources are stored as `SED` instances (such as the one above) within the `SourceCatalog.sources` attribute.

In [61]:
# Get a source instance
source1 = cat.get_SED(0)

# Calculate the results
print(source1.results)
print('\nInput model:', source1.Teff_model, source1.logg_model, source1.FeH_model)

# Plot it
fig = source1.plot(spectra='all')

    param      value     unc       units    
------------- -------- ------- -------------
         name Source 5      --            --
          age      6.0     4.0           Gyr
     distance       --      --            --
     parallax       --      --            --
       radius       --      --            --
          SpT       --      --            --
      SpT_fit  Opt:M7V      --            --
spectral_type       --      --            --
   membership       --      --            --
         fbol 1.89e-11 4.7e-14 erg / (cm2 s)
         mbol   15.327   0.003            --
         Lbol       --      --            --
     Lbol_sun       --      --            --
         Mbol       --      --            --
         logg       --      --            --
         mass       --      --            --
         Teff       --      --            --
      Teff_bb       --      --            --
     Teff_evo       --      --            --

Input model: 2700 4.0 0.5


Hooray! Let's take a look at another one.

In [62]:
# Get the source instance
source2 = cat.get_SED(1)

# Calculate the results
print(source2.results)
print('\nInput model:', source2.Teff_model, source2.logg_model, source2.FeH_model)

# Plot it
fig = source2.plot(spectra='all')

    param      value     unc        units    
------------- -------- -------- -------------
         name Source 7       --            --
          age      6.0      4.0           Gyr
     distance       --       --            --
     parallax       --       --            --
       radius       --       --            --
          SpT       --       --            --
      SpT_fit  Opt:M5V       --            --
spectral_type       --       --            --
   membership       --       --            --
         fbol 1.89e-11 4.53e-14 erg / (cm2 s)
         mbol   15.324    0.003            --
         Lbol       --       --            --
     Lbol_sun       --       --            --
         Mbol       --       --            --
         logg       --       --            --
         mass       --       --            --
         Teff       --       --            --
      Teff_bb       --       --            --
     Teff_evo       --       --            --

Input model: 3500 3.0 0.5


Fantastic!

Let's see a table of the results and plot all the SEDs!

In [63]:
cat.results[['name','spectral_type','fbol','radius','Teff_bb','SpT_fit']]

name,spectral_type,fbol,radius,Teff_bb,SpT_fit
,,erg / (cm2 s),solRad,K,
object,object,object,object,object,object
Source 5,None,1.890049217043731e-11,None,None,Opt:M7V
Source 7,None,1.8946528824006648e-11,None,None,Opt:M5V
Source 11,None,3.2944384012031384e-12,None,None,Opt:M4V
Source 16,None,1.0066053621695187e-11,None,None,Opt:M4V
Source 17,None,2.012373054020361e-12,None,None,Opt:M5V


In [64]:
# We imposed a color cut of NIRISS.F150W-NIRISS.F200W in (0.3, 0.5) and NIRISS.F158M-NIRISS.F150W in (-0.2, -0.1)
# Let's see if it worked
color = cat.plot('NIRISS.F150W-NIRISS.F200W', 'NIRISS.F158M-NIRISS.F150W')
show(color)

Nice!